<center>
<h3></h3>
<img src="https://github.com/always-further/deepfabric/raw/main/assets/logo-light-hols.png" width="25%">
<a href="https://alwaysfurther.ai"><h5>By Always Further AI</a></h5>
<h3>Train and Evaluate Qwen3-4B-Thinking as a Blender MCP Agent</h3>
<!-- CTA Buttons -->
<p>
  <a href="https://github.com/always-further/deepfabric/issues?q=is%3Aissue+is%3Aopen+label%3A%22good+first+issue%22">
    <img src="https://img.shields.io/badge/Contribute-Good%20First%20Issues-green?style=for-the-badge&logo=github" alt="Good First Issues"/>
  </a>
  &nbsp;
  <a href="https://discord.gg/pPcjYzGvbS">
    <img src="https://img.shields.io/badge/Chat-Join%20Discord-7289da?style=for-the-badge&logo=discord&logoColor=white" alt="Join Discord"/>
  </a>
</p>
<!-- Badges -->
<p>
  <a href="https://opensource.org/licenses/Apache-2.0">
    <img src="https://img.shields.io/badge/License-Apache%202.0-blue.svg" alt="License"/>
  </a>
  <a href="https://github.com/always-further/deepfabric/actions/workflows/test.yml">
    <img src="https://github.com/always-further/deepfabric/actions/workflows/test.yml/badge.svg" alt="CI Status"/>
  </a>
  <a href="https://pypi.org/project/deepfabric/">
    <img src="https://img.shields.io/pypi/v/deepfabric.svg" alt="PyPI Version"/>
  </a>
  <a href="https://pepy.tech/project/deepfabric">
    <img src="https://static.pepy.tech/badge/deepfabric" alt="Downloads"/>
  </a>
  <a href="https://discord.gg/pPcjYzGvbS">
    <img src="https://img.shields.io/discord/1384081906773131274?color=7289da&label=Discord&logo=discord&logoColor=white" alt="Discord"/>
  </a>
  <a href="https://www.reddit.com/r/deepfabric/">
    <img src="https://img.shields.io/badge/Reddit-r%2Fdeepfabric-FF4500?logo=reddit&logoColor=white" alt="Reddit"/>
  </a>
</p>
</center>

This notebook demonstrates using DeepFabric and Unsloth / Transformers and TRL to train a 4 billion parameter model to out perform a frontier labs model at Tool Calling- using the particularly difficult and unique [Blender MCP server](https://github.com/ahujasid/blender-mcp).

From our initial tests with just smaller dataset of less than 1k, we saw the DeepFabric Fine-Tuned model outperform both Gemini 2.5 pro and Claude Sonnet 4.5

| Model | Score |
|-------|-------|
| DeepFabric Fine Tuned | 93.50% |
| Claude Sonnet 4.5 | 80.50% |
| Google Gemini Pro 2.5 | 47.00% |

DeepFabric is a tool to generate synthetic data for training language models along with a post-train evaluation suite. By combining reasoning traces with tool-calling patterns, it creates high-quality, domain-specific datasets that teach models to think, plan, and call tools correctly, while conforming to strict schema structures.

What sets DeepFabric apart from other dataset generation tools is its ability to ensure high diversity yet domain-anchored relevance through unique topic graph generation algorithms, while using live isolation Tool traces to force the model to deal with hallunications and other failures. This guides sample creation to cover all necessary subtopics while avoiding redundancy, which is where other tools often fall short, resulting in model overfit.

DeepFabric fine tuned models on specific tools / functions and knowledge domains, will very often outperform frontier models.

#### **Free as in Beer 🍻**
You can run this on a free T4 instance - just select ***Runtime → Run all***.

You’re welcome to copy this Colab notebook, create your own agents, and share it with friends and family.

DeepFabric is open source and completely unrestricted, so knock yourself out!

It really helps us know we’re providing value, so if you could ⭐ star us on [Github](https://github.com/always-further/deepfabric) ⭐ that would be ace!

#### **Standing on the shoulders of (un)Sloths!**

A lot of this training book is inherited from the incredible work carried out by the awesome Unsloth.ai team -  ⭐️ <i>Star them on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>


## **Dataset Generation**

![DeepFabric Terminal Demo](https://github.com/always-further/deepfabric/blob/main/assets/df-demo.gif?raw=true)

Two choices available here, you can either use the existing dataset we have generated or create your own.

Creating your own is a little more involved, but obviously a lot more useful, as you will learn something you can apply over and over, enabling you to create your own custom trained models.

The dataset generation will take place outside of the notebook, as we will use isolated live tool traces which vastly increase dataset quality, **so the following should be actioned outside of this notebook on a machine capable of running docker**

1. **Run the DeepFabric tools-sdk docker image**

`docker run -d -p 3000:3000 ghcr.io/always-further/deepfabric/tools-sdk:latest`

2. **Create the mock Tools schema from the Blender MCP Workspace Server**

```bash
deepfabric import-tools --transport stdio \
      --command "uvx blender-mcp" \
      --spin http://localhost:3000
```


3. **Download the following tool data we prepared and the deepfabric config file used for generation**

`curl -O  https://gist.githubusercontent.com/lukehinds/6f7ce825c0595103d6ff39c9a6f4cdee/raw/blender-mock-data.json`

`curl -O  https://gist.githubusercontent.com/lukehinds/fb6ee66a33c73cca43ce00a0d73fac42/raw/spin-blender.yaml`

`curl -O  https://gist.githubusercontent.com/lukehinds/ca224193f25e69487640be46af87b807/raw/load-blender-mock-data.sh`


3. **Run the load data script**

The following will load lots of mocked data into the DeepFabric Tools container, that will provide realistic responses to tool calls.

```bash
./load-blender-mock-data.sh
```

4. **Verify the data loaded correctly:**

```
curl -s -X POST http://localhost:3000/mock/execute -H 'Content-Type: application/json' -d '{"name": "get_viewport_screenshot", "arguments": {"filePath": "/tmp/viewport.png"}}' | jq
{
  "result": {
    "cameraView": false,
    "filePath": "/tmp/viewport.png",
    "format": "PNG",
    "resolution": {
      "height": 1080,
      "width": 1920
    },
    "size": 3145728,
    "success": true,
    "viewportShading": "MATERIAL"
  }
}
```

5. **Generate your dataset**

You can now generate your dataset using the provided config:

`deepfabric generate spin-blender.yaml`

> **_NOTE:_**  The supplied config is setup to use OpenAI and gpt-41 as a model, but you can easily overwrite to use your prefered model as DeepFabric supports Gemini, OpenAI, Anthropic, Ollama and Openrouter, just remember to export your API KEY!

So for OpenAI:

`export OPENAI_API_KEY="sk-proj-xxxx`

`deepfabric generate spin-blender.yaml --provider openai --model gpt-4o`

The config will generate 100 samples.

6. **Push to HuggingFace**

`deepfabric upload blender-dataset.jsonl --repo org/deepfabric-blender-mcp-workspace-dataset --token <your-huggingface-token>`


### **2. Install Dependencies**

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth unsloth_zoo
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.57.3
!pip install --no-deps trl==0.22.2
!pip install deepfabric

### **3. Imports and Setup**

In [ ]:
import json
from pathlib import Path
from datasets import Dataset, load_dataset, DatasetDict
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


> **_NOTE:_**  Remember to update your dataset repo to `HF_DATASET_NAME`! or just leave as **`alwaysfurther/deepfabric-blender-mcp`** if you did not generate your own dataset.

In [ ]:
# Configuration
HF_DATASET_NAME = "alwaysfurther/deepfabric-blender-mcp"
HF_DATASET_SPLIT = "train"  # Split to load (usually 'train' for full dataset)
MODEL_NAME = "unsloth/Qwen3-4B-Thinking-2507"
MAX_SEQ_LENGTH = 4096 # we have some thinking traces with calls.
REQUIRE_THINKING = True  # Filter out samples without thinking traces

# Think tag configuration
THINK_TAG_OPEN = "<think>"
THINK_TAG_CLOSE = "</think>"

# Split ratios (must sum to 1.0)
TRAIN_RATIO = 0.8 # For SFT Training
EVAL_RATIO = 0.1  # For Training evaluation
TEST_RATIO = 0.1  # For DeepFabric evaluation

### **4. Load Tokenizer**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

print(f"Tokenizer loaded: {MODEL_NAME}")
print(f"Has chat_template: {hasattr(tokenizer, 'chat_template') and tokenizer.chat_template is not None}")

==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Tokenizer loaded: unsloth/Qwen3-4B-Thinking-2507
Has chat_template: True


### **5. Load and Configura Lora**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.9 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### **5. Load Dataset**

In [ ]:
# Load dataset from HuggingFace Hub
dataset = load_dataset(HF_DATASET_NAME, split=HF_DATASET_SPLIT)

print(f"Loaded {len(dataset)} samples from {HF_DATASET_NAME}")
print(f"Fields: {dataset.column_names}")

README.md:   0%|          | 0.00/100 [00:00<?, ?B/s]

data/train.jsonl:   0%|          | 0.00/24.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1118 [00:00<?, ? examples/s]

Loaded 1118 samples from alwaysfurther/deepfabric-blender-mcp
Fields: ['messages', 'metadata', 'reasoning', 'tool_context', 'tools', 'agent_context', 'question', 'final_answer']


### 7. **Train/Eval/Test Split**

Create three splits:
- **Train**: For SFT training (80%)
- **Eval**: For validation during training (10%)
- **Test**: For DeepFabric evaluation after training (10%)

Use `prepare_dataset_for_training` to strip to only the Tools used within a single sample

In [ ]:
from deepfabric.training import prepare_dataset_for_training

# SPLIT FIRST - before any filtering so we have a full tool set for eval
split = dataset.train_test_split(test_size=0.2, seed=42)
train_raw = split["train"]  # 80%

test_split = split["test"].train_test_split(test_size=0.5, seed=42)
eval_raw = test_split["train"]   # 10%
test_ds = test_split["test"]     # 10% - KEEP ALL TOOLS for evaluation

# Filter ONLY training data
train_ds = prepare_dataset_for_training(train_raw, tool_strategy="used_only")
eval_ds = prepare_dataset_for_training(eval_raw, tool_strategy="used_only")

print(f"\n=== SPLIT RESULTS ===")
print(f"Train: {len(train_ds)} samples, {len(train_ds[0]['tools'])} tools (filtered)")
print(f"Eval:  {len(eval_ds)} samples, {len(eval_ds[0]['tools'])} tools (filtered)")
print(f"Test:  {len(test_ds)} samples, {len(test_ds[0]['tools'])} tools (ALL tools)")


Filtering tools:   0%|          | 0/894 [00:00<?, ? examples/s]

Filtering tools:   0%|          | 0/112 [00:00<?, ? examples/s]


=== SPLIT RESULTS ===
Train: 894 samples, 4 tools (filtered)
Eval:  112 samples, 1 tools (filtered)
Test:  112 samples, 21 tools (ALL tools)


In [ ]:
def format_example(example):
    messages = [{k: v for (k, v) in msg.items() if v is not None} for msg in example['messages']]
    tools = example.get('tools', None)

    reasoning = example.get('reasoning', {})
    reasoning_content = reasoning.get('content', [])

    # Create mapping of step_number to thought
    step_thoughts = {}
    for step in reasoning_content:
        step_num = step.get('step_number')
        thought = step.get('thought', '')
        if step_num is not None and thought:
            step_thoughts[step_num] = thought

    # Assign reasoning_content to each assistant message
    assistant_count = 0
    for i, msg in enumerate(messages):
        if msg.get('role') == 'assistant':
            assistant_count += 1
            thought = step_thoughts.get(assistant_count, '')
            if thought:
                messages[i]['reasoning_content'] = thought

    text = tokenizer.apply_chat_template(
        messages,
        tools=tools,
        tokenize=False
    )

    import re
    text = re.sub(r'<think>\s*</think>\s*', '', text)

    return {'text': text}

# Format training and eval (filtered - smaller sequences)
train_formatted = train_ds.map(format_example)
eval_formatted = eval_ds.map(format_example)

# test_ds stays raw - pass directly to evaluator (has all 21 tools)


Map:   0%|          | 0/894 [00:00<?, ? examples/s]

Map:   0%|          | 0/112 [00:00<?, ? examples/s]

In [ ]:
# Train on responses only - train on the assistant outputs and ignore the loss on the user's inputs.
# This helps increase accuracy of finetunes (source: unsloth)
def train_on_responses_only(example, tokenizer):
    text = example['text']
    encoding = tokenizer(
        text,
        add_special_tokens=False,
        return_offsets_mapping=True,
        max_length=8192,  # Qwen3 supports up to 32k, 16k is safe
        truncation=True,
    )

    input_ids = encoding['input_ids']
    offset_mapping = encoding['offset_mapping']

    # Start with all masked
    labels = [-100] * len(input_ids)

    # Find all assistant response sections in the original text
    # Pattern: <|im_start|>assistant\n ... <|im_end|>
    import re
    pattern = r'<\|im_start\|>assistant\n(.*?)<\|im_end\|>'

    # Find character spans of assistant content
    assistant_spans = []
    for match in re.finditer(pattern, text, re.DOTALL):
        # Get the span of the actual content (group 1), not the tags
        start_char = match.start(1)
        end_char = match.end(1)
        assistant_spans.append((start_char, end_char))

    # Unmask tokens that fall within assistant spans
    for token_idx, (token_start, token_end) in enumerate(offset_mapping):
        if token_start is None or token_end is None:
            continue

        # Check if this token is inside any assistant span
        for span_start, span_end in assistant_spans:
            if token_start >= span_start and token_end <= span_end:
                labels[token_idx] = input_ids[token_idx]
                break

    return {
        'input_ids': input_ids,
        'labels': labels,
        'attention_mask': [1] * len(input_ids),
    }


# Apply to dataset
tokenized_dataset = train_formatted.map(
    lambda x: train_on_responses_only(x, tokenizer),
    remove_columns=train_formatted.column_names,
    num_proc=16,
)

tokenized_eval_ds = eval_formatted.map(
    lambda x: train_on_responses_only(x, tokenizer),
    remove_columns=eval_formatted.column_names,
    num_proc=16,
)

# Verify it worked
sample = tokenized_dataset[50]

# Show what we're training on
trained_ids = [id for id, label in zip(sample['input_ids'], sample['labels']) if label != -100]
print("Training on this text:")
print(tokenizer.decode(trained_ids))

Map (num_proc=16):   0%|          | 0/894 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/112 [00:00<?, ? examples/s]

Training on this text:
<think>
The user wants to apply a procedural noise texture to the 'Mountain' object. Before attempting to apply the texture, I need to verify that the 'Mountain' object exists in the Blender scene. I will use the `get_object_info` tool for this purpose.
</think>

<tool_call>
{"name": "get_object_info", "arguments": {"object_name":"Mountain"}}
</tool_call><think>
The user wants to apply a procedural noise texture to the 'Mountain' object. I have confirmed that the 'Mountain' object exists. Now I need to execute Blender Python code to create a new material, add a noise texture, and apply it to the object. I will start by selecting the object and ensuring it has a material slot.
</think>

<tool_call>
{"name": "execute_blender_code", "arguments": {"code":"import bpy\n\nobj = bpy.data.objects.get('Mountain')\nif obj:\n    bpy.context.view_layer.objects.active = obj\n    obj.select_set(True)\n\n    # Ensure the object has a material slot\n    if not obj.data.materials:

### 6. **Calculate Max Sequence Length**

We can use this to optimize training on `max_seq_length`

In [ ]:
# Calculate token lengths for training config
def get_token_length(text):
    return len(tokenizer.encode(text))

# Sample to estimate max length (or compute on full dataset)
sample_size = min(1000, len(train_formatted))
lengths = [get_token_length(train_formatted[i]["text"]) for i in range(sample_size)]

max_length = max(lengths)
avg_length = sum(lengths) / len(lengths)

# Add buffer
RECOMMENDED_MAX_LENGTH = max_length + 100

print(f"Token length stats (sample of {sample_size}):")
print(f"  Max: {max_length}")
print(f"  Avg: {avg_length:.0f}")
print(f"  Recommended max_length for training: {RECOMMENDED_MAX_LENGTH}")

Token length stats (sample of 894):
  Max: 4827
  Avg: 1563
  Recommended max_length for training: 4927


##### **Preview Formatted Output**

In [ ]:
# Preview a formatted example (truncated)
MAX_PREVIEW_CHARS = 2000

if len(train_ds) > 0:
    text = train_formatted[0]["text"]

    print("=" * 80)
    print("FORMATTED TEXT:")
    print("=" * 80)
    if len(text) > MAX_PREVIEW_CHARS:
        print(text[:MAX_PREVIEW_CHARS])
        print(f"\n... [truncated, {len(text)} total chars]")
    else:
        print(text)
else:
    print("No samples in training set")


FORMATTED TEXT:
<|im_start|>system
# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"function": {"description": "\n    Check if Hyper3D Rodin integration is enabled in Blender.\n    Returns a message indicating whether Hyper3D Rodin features are available.\n\n    Don't emphasize the key type in the returned message, but sliently remember it. \n    ", "name": "get_hyper3d_status", "parameters": {"properties": null, "required": null, "type": "object"}}, "type": "function"}
{"function": {"description": "\n    Check if Hunyuan3D integration is enabled in Blender.\n    Returns a message indicating whether Hunyuan3D features are available.\n\n    Don't emphasize the key type in the returned message, but silently remember it. \n    ", "name": "get_hunyuan3d_status", "parameters": {"properties": null, "required": null, "type": "object"}}, "type": "function"}
{"function": {"description

##### **Verify think tags are present**

In [ ]:

if len(train_ds) > 0:
    text = train_formatted[0]["text"]
    has_think_open = THINK_TAG_OPEN in text
    has_think_close = THINK_TAG_CLOSE in text

    print(f"Think tag open: {has_think_open}")
    print(f"Think tag close: {has_think_close}")

    if has_think_open and has_think_close:
        start = text.find(THINK_TAG_OPEN) + len(THINK_TAG_OPEN)
        end = text.find(THINK_TAG_CLOSE)
        think_content = text[start:end]
        print(f"\nThink content preview: {think_content[:200]}...")


Think tag open: True
Think tag close: True

Think content preview: 
The user wants a 3D model of a whimsical treehouse. I need to check which 3D generation tools are available to fulfill this request. I will start by checking the status of Hyper3D and Hunyuan3D.
...


## 8. **Lora Based SFT with Unsloth**

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_formatted,
    eval_dataset=eval_formatted,
    args=SFTConfig(
        dataset_text_field = "text",
        output_dir = "./output",
        max_seq_length=RECOMMENDED_MAX_LENGTH,
        per_device_train_batch_size = 2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps = 8,
        warmup_ratio=0.03,

        # Eval for best model checkpoint
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",

        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/894 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/112 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 894 | Num Epochs = 3 | Total steps = 168
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
100,0.314300,0.316839


Unsloth: Not an error, but Qwen3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
# @title Show time stats
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)

7369.6085 seconds used for training.
122.83 minutes used for training.


#### **Save Lora Adapters**

In [ ]:
model.save_pretrained("lora")
tokenizer.save_pretrained("lora")
# model.push_to_hub("your_name/finetune", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/finetune", token = "...") # Online saving

('lora/tokenizer_config.json',
 'lora/special_tokens_map.json',
 'lora/chat_template.jinja',
 'lora/vocab.json',
 'lora/merges.txt',
 'lora/added_tokens.json',
 'lora/tokenizer.json')

## 9. **DeepFabric Evaluation**

DeepFabric evaluations works in the following way, during dataset configuration we split off 10% of the dataset reserved for evaluations, this means that the model was not trained on the evaluation dataset, so it will be blind to the users question and correct answer. We only run 10 evals here, but you can extend to more should you wish, just increase `max_samples`

Scores will be based on the model calling the correct tool e.g. `"get_object_info"` `"arguments": {"object_name":"Mountain"}}`

Note, evaluation can be on the slow side using transformers as a backend. If this is too long a wait, you can just save and push a GGUF and then run evaluation with an ollama backend.

For the evaluation we provide the full set of tools that the model needs to select from with the `test_ds` split.

```python
config = EvaluatorConfig(
    inference_config=InferenceConfig(
        model="org/finetune-gguf",
        backend="ollama",
        load_in_4bit=True,
    ),
...
)
```

**Note**: Response Quality will be 0%, but is not used for tool evaluation. It is a placeholder for when we introduce semantic checks later on.


In [ ]:
from deepfabric.evaluation import Evaluator, EvaluatorConfig, InferenceConfig

config = EvaluatorConfig(
    inference_config=InferenceConfig(
        model=model,
        tokenizer=tokenizer,
        backend="transformers",
        temperature=0.1,
        max_tokens=RECOMMENDED_MAX_LENGTH,
    ),
    max_samples=10,
    save_predictions=True,
    multi_turn=False,
    output_path="trained_results.json",
)

# Run evaluation
evaluator = Evaluator(config)
results = evaluator.evaluate(dataset=test_ds)

# Print summary
evaluator.print_summary(results.metrics)
# Cleanup GPU memory
evaluator.cleanup()

Loading dataset...

Loaded 10 samples

Running evaluation...

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluation complete!

...

Tool Selection Accuracy: 90.00%

Parameter Accuracy: 100.00%

Execution Success Rate: 90.00%

Response Quality: 0.00%

Overall Score: 93.50%

## **Eval against Frontier models**

You can also run evals against any API based model to help get a good comparision - support is available for:

* `openai`
* `openrouter`
* `gemini`

Just remember to add your api key!

```python
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
```

In this instance we will run against `gemini-2.5-flash` which our tuned model should outperform. Keep in mind, this is using a dataset on the smaller side (less than 1k), with something more substantial 10k to 50k tuned models will often hit a 100% success rate during evaluation.

# Gemini Pro 2.5
<img src="https://apipie.ai/docs/img/Models/Gemini.png" alt="drawing" width="200"/>

In [ ]:
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from deepfabric.evaluation import Evaluator, EvaluatorConfig, InferenceConfig

# Configure evaluator for Gemini
config = EvaluatorConfig(
    inference_config=InferenceConfig(
        backend="llm",
        provider="gemini",
        model="gemini-2.5-pro",
        max_tokens=RECOMMENDED_MAX_LENGTH,
        temperature=0.1,
        api_key=api_key,
    ),
    max_samples=10,
    save_predictions=True,
    multi_turn=False,
    output_path="gemini-2.5-pro.json",
)

# Run evaluation
evaluator = Evaluator(config)
results = evaluator.evaluate(dataset=test_ds)

# Print summary
evaluator.print_summary(results.metrics)
# Cleanup GPU memory
evaluator.cleanup()

Loading dataset...

Loaded 10 samples

Running evaluation...

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluation complete!

...

Tool Selection Accuracy: 40.00%

Parameter Accuracy: 60.00%

Execution Success Rate: 40.00%

Response Quality: 0.00%

Overall Score: 47.00%

# Claude Sonnet 4.5
<img src="https://diplo-media.s3.eu-central-1.amazonaws.com/2025/09/Anthropic-Claude-Sonnet-4.5-Claude-Code-Computer-Use-AI-coding-features-1024x576.jpg" alt="drawing" width="200"/>

In [ ]:
from google.colab import userdata
claude_api_key = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from deepfabric.evaluation import Evaluator, EvaluatorConfig, InferenceConfig

# Configure evaluator for Gemini
config = EvaluatorConfig(
    inference_config=InferenceConfig(
        backend="llm",
        provider="anthropic",
        model="claude-sonnet-4-5",
        max_tokens=RECOMMENDED_MAX_LENGTH,
        temperature=0.1,
        api_key=claude_api_key,
    ),
    max_samples=10,
    save_predictions=True,
    multi_turn=False,
    output_path="claude-sonnet-4-5.json",
)

# Run evaluation
evaluator = Evaluator(config)
results = evaluator.evaluate(dataset=test_ds)

# Print summary
evaluator.print_summary(results.metrics)
# Cleanup GPU memory
evaluator.cleanup()

Loading dataset...

Loaded 10 samples

Running evaluation...

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluation complete!

...

Tool Selection Accuracy: 70.00%

Parameter Accuracy: 100.00%

Execution Success Rate: 70.00%

Response Quality: 0.00%

Overall Score: 80.50%

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "finetune-gguf",
        tokenizer,
        quantization_method = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token to the notebook secrets as `HF_TOKEN` and upload location!

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "org/deepfabric-blender-mcp-gguf",
        tokenizer,
        quantization_method = "Q8_0", # Only Q8_0, BF16, F16 supported
        token = hf_token,
    )

## **Wrapping up**

What next, we recommend reading the [docs](https://docs.deepfabric.dev) and trying out your own model creations.

Anything you do cook up, **please do share** with the community on our [Reddit](https://www.reddit.com/r/deepfabric/) or in our [Discord server](https://discord.com/invite/pPcjYzGvbS)

DeepFabric is open source and completely unrestricted, so knock yourself out!

Don't forget to give us a ⭐ star us on [Github](https://github.com/always-further/deepfabric) ⭐ if this has been useful to you.